# Load PDF

In [2]:
from langchain_community.document_loaders import(
PyPDFLoader,
PyMuPDFLoader,
UnstructuredPDFLoader
)

In [ ]:
# pdf loader

try:
    pypdf_loader = PyPDFLoader("data/pdf/attention.pdf")
    pypdf_docs = pypdf_loader.load()
    print(pypdf_docs)

except Exception as e:
    print(f"error :{e}")